# Loadings

In [ ]:
#!pip install torch torchvision transformers opencv-python pandas ultralytics moviepy ollama pytubefix
#!pip show moviepy

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pytubefix import YouTube
from pytubefix.cli import on_progress
import pandas as pd
from moviepy.video.io.VideoFileClip import VideoFileClip
from PIL import Image
import io
from tqdm import tqdm
import os
import ollama

In [2]:
# Load previously created csv with videos and video details
df = pd.read_csv('Video_Details.csv', encoding='ISO-8859-1')

In [10]:
df.head(2)

,channel_name,video_id,video_title,published_datetime,duration,view_count,like_count,dislike_count,comment_count,description,thumbnail_url
0,@Mrwhosetheboss,neIYdLysqlk,I tested the Craziest Xiaomi Gadgets!,2024-11-13T12:04:54Z,PT27M53S,6679521,219704,0,8890,I'm still genuinely in shock how great some of...,https://i.ytimg.com/vi/neIYdLysqlk/hqdefault.jpg
1,@Mrwhosetheboss,YX8ks42Azn8,The TRIPLE FOLDING phone has a Problem.,2024-10-26T14:06:50Z,PT12M54S,3464013,111736,0,5932,This phone has a LOT of good....and a LOT of b...,https://i.ytimg.com/vi/YX8ks42Azn8/hqdefault.jpg


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   channel_name        45 non-null     object
 1   video_id            45 non-null     object
 2   video_title         45 non-null     object
 3   published_datetime  45 non-null     object
 4   duration            45 non-null     object
 5   view_count          45 non-null     int64 
 6   like_count          45 non-null     int64 
 7   dislike_count       45 non-null     int64 
 8   comment_count       45 non-null     int64 
 9   description         45 non-null     object
 10  thumbnail_url       45 non-null     object
dtypes: int64(4), object(7)
memory usage: 4.0+ KB


In [11]:
# Create video urls
df['video_url'] = df['video_id'].apply(lambda x: f"https://www.youtube.com/watch?v={x}")

In [6]:
df.head(1)

,channel_name,video_id,video_title,published_datetime,duration,view_count,like_count,dislike_count,comment_count,description,thumbnail_url,video_url
0,@Mrwhosetheboss,neIYdLysqlk,I tested the Craziest Xiaomi Gadgets!,2024-11-13T12:04:54Z,PT27M53S,6679521,219704,0,8890,I'm still genuinely in shock how great some of...,https://i.ytimg.com/vi/neIYdLysqlk/hqdefault.jpg,https://www.youtube.com/watch?v=neIYdLysqlk


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   channel_name        45 non-null     object
 1   video_id            45 non-null     object
 2   video_title         45 non-null     object
 3   published_datetime  45 non-null     object
 4   duration            45 non-null     object
 5   view_count          45 non-null     int64 
 6   like_count          45 non-null     int64 
 7   dislike_count       45 non-null     int64 
 8   comment_count       45 non-null     int64 
 9   description         45 non-null     object
 10  thumbnail_url       45 non-null     object
 11  video_url           45 non-null     object
dtypes: int64(4), object(8)
memory usage: 4.3+ KB


In [10]:
test_df = df.head(1)

# Unified Code

In [20]:
# Helper function to extract frames from a video
# def extract_frames(video_path, frame_rate=1):
#     clip = VideoFileClip(video_path)
#     fps = clip.fps
#     frame_list = []
#     timestamps = []

#     for i, frame in enumerate(clip.iter_frames()):
#         if i % int(fps / frame_rate) == 0:
#             image = Image.fromarray(frame)
#             buffer = io.BytesIO()
#             image.save(buffer, format="JPEG")
#             buffer.seek(0)
#             frame_list.append(buffer)
#             timestamps.append(i / fps)

#     return frame_list, timestamps

def extract_frames(video_path, num_frames=25):
    clip = VideoFileClip(video_path)
    duration = clip.duration  # Total duration of the video in seconds
    timestamps = np.linspace(0, duration, num_frames, endpoint=False)  # Generate 25 evenly spaced timestamps
    frame_list = []

    for timestamp in timestamps:
        frame = clip.get_frame(timestamp)  # Extract frame at the specific timestamp
        image = Image.fromarray(frame)
        buffer = io.BytesIO()
        image.save(buffer, format="JPEG")
        buffer.seek(0)
        frame_list.append(buffer)

    return frame_list, timestamps

# Object detection using LLaMA
def detect_objects(input_frames):
    message_list = []
    for frame in tqdm(input_frames, desc="Detecting Objects in Frames"):
        try:
            # Read the BytesIO object into raw bytes
            frame_bytes = frame.getvalue()
            
            # Use Ollama to send the frame for object detection
            res = ollama.chat(
                model="llava",
                messages=[
                    {
                        "role": "user",
                        "content": "Describe all objects seen in this image.",
                        "images": [frame_bytes],  # Pass raw bytes
                    }
                ],
            )
            message_list.append(res['message']['content'])
        except Exception as e:
            print(f"Failed to process frame: {e}")
            message_list.append("Error processing frame")
    return message_list

# Main function to analyze a single video
def analyze_video(video_path):
    print(f"Analyzing video: {video_path}...")
    frames, timestamps = extract_frames(video_path, num_frames=25)
    print("Running object detection...")
    object_data = detect_objects(frames)

    # Combine data into a DataFrame
    data = {
        "Timestamp": timestamps,
        "Detected Objects": object_data,
    }
    df = pd.DataFrame(data)
    return df

# Function to download and analyze all videos in the DataFrame
# def download_and_analyze_videos(df):
#     os.makedirs("videos_downloaded", exist_ok=True)
#     all_results = []

#     for index, row in tqdm(df.iterrows(), total=len(df), desc="Downloading Videos"):
#         try:
#             url = row['video_url']
#             video_id = row['video_id']
#             download_path = "videos_downloaded"
#             output_filename = f"{video_id}.mp4"

#             # Download the video
#             yt = YouTube(url, on_progress_callback=on_progress)
#             ys = yt.streams.get_highest_resolution()
#             ys.download(output_path=download_path, filename=output_filename)

#             # Analyze the downloaded video
#             video_path = os.path.join(download_path, output_filename)
#             analysis_results = analyze_video(video_path)

#             # Save the analysis results
#             output_csv = os.path.join("videos_downloaded", f"{video_id}_analysis.csv")
#             analysis_results.to_csv(output_csv, index=False)
#             print(f"Analysis saved for {video_id} at {output_csv}")
        
#         except Exception as e:
#             print(f"Failed to process video {row['video_url']}: {e}")
        
#         # Combine all results into a single DataFrame
#         comprehensive_df = pd.concat(all_results, ignore_index=True)
    
#         # Save the comprehensive DataFrame as a single CSV
#         comprehensive_csv_path = os.path.join("videos_downloaded", "comprehensive_analysis.csv")
#         comprehensive_df.to_csv(comprehensive_csv_path, index=False)
#         print(f"Comprehensive analysis saved at {comprehensive_csv_path}")

def download_and_analyze_videos(df):
    os.makedirs("videos_downloaded", exist_ok=True)
    all_results = []  # List to store DataFrames for all videos

    for index, row in tqdm(df.iterrows(), total=len(df), desc="Downloading and Analyzing Videos"):
        try:
            url = row['video_url']
            video_id = row['video_id']
            download_path = "videos_downloaded"
            output_filename = f"{video_id}.mp4"

            # Download the video
            yt = YouTube(url, on_progress_callback=on_progress)
            video_title = yt.title
            ys = yt.streams.get_highest_resolution()
            ys.download(output_path=download_path, filename=output_filename)

            # Analyze the downloaded video
            video_path = os.path.join(download_path, output_filename)
            analysis_results = analyze_video(video_path)

            # Add video_id to the results
            analysis_results['video_id'] = video_id
            analysis_results['video_url'] = url
            analysis_results['video_title'] = video_title

            # Append results to the list
            all_results.append(analysis_results)

        except Exception as e:
            print(f"Failed to process video {row['video_url']}: {e}")

    # Combine all results into a single DataFrame
    comprehensive_df = pd.concat(all_results, ignore_index=True)
    
    # Save the comprehensive DataFrame as a single CSV
    comprehensive_csv_path = os.path.join("videos_downloaded", "comprehensive_analysis.csv")
    comprehensive_df.to_csv(comprehensive_csv_path, index=False)
    print(f"Comprehensive analysis saved at {comprehensive_csv_path}")

In [21]:
# Run the workflow
download_and_analyze_videos(df)

Analyzing video: videos_downloaded\neIYdLysqlk.mp4...
Running object detection...



Detecting Objects in Frames: 100%|██████████| 25/25 [01:11<00:00,  2.87s/it]

Analyzing video: videos_downloaded\YX8ks42Azn8.mp4...
Running object detection...



Detecting Objects in Frames: 100%|██████████| 25/25 [01:17<00:00,  3.09s/it]

Analyzing video: videos_downloaded\4RcThoRG46c.mp4...
Running object detection...


...

Analyzing video: videos_downloaded\yro-CI984mE.mp4...
Running object detection...



Detecting Objects in Frames: 100%|██████████| 25/25 [01:47<00:00,  4.29s/it]

Analyzing video: videos_downloaded\w9EbwjnpKaA.mp4...
Running object detection...



Detecting Objects in Frames: 100%|██████████| 25/25 [01:56<00:00,  4.65s/it]

Comprehensive analysis saved at videos_downloaded\comprehensive_analysis.csv
